In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('/content/drive/MyDrive/CodePractice/10. Hyperparameter/diabetes.csv')

In [3]:
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


There are lots of 0 values in col you can see like insulin is 0 which is not possible so we can replce this values with mean. In DL is not major problem but in Ml it is big prblem

In [3]:
df  = data.copy()

In [5]:
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [7]:
df.corr()["Outcome"]

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [4]:
x = df.iloc[:, :-1]
y = df.iloc[:,-1]

In [5]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x = scaler.fit_transform(x)

In [6]:
# split the data into training and test
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=101)

# build model now

In [7]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout

In [8]:
model = Sequential()
model.add(Dense(units = 32, activation = "relu", input_dim = 8))
model.add(Dense(1,activation = "sigmoid"))
model.compile(optimizer="adam", loss = "binary_crossentropy", metrics= ["accuracy"])

In [9]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), batch_size=32, epochs=10)

Epoch 1/10
20/20 [==============================] - 1s 17ms/step - loss: 0.8086 - accuracy: 0.3567 - val_loss: 0.7438 - val_accuracy: 0.3571
Epoch 2/10
20/20 [==============================] - 0s 6ms/step - loss: 0.7355 - accuracy: 0.4528 - val_loss: 0.6863 - val_accuracy: 0.5584
Epoch 3/10
20/20 [==============================] - 0s 5ms/step - loss: 0.6800 - accuracy: 0.5814 - val_loss: 0.6424 - val_accuracy: 0.6494
Epoch 4/10
20/20 [==============================] - 0s 4ms/step - loss: 0.6370 - accuracy: 0.6580 - val_loss: 0.6086 - val_accuracy: 0.6948
Epoch 5/10
20/20 [==============================] - 0s 4ms/step - loss: 0.6031 - accuracy: 0.7134 - val_loss: 0.5823 - val_accuracy: 0.6948
Epoch 6/10
20/20 [==============================] - 0s 4ms/step - loss: 0.5746 - accuracy: 0.7166 - val_loss: 0.5605 - val_accuracy: 0.6948
Epoch 7/10
20/20 [==============================] - 0s 4ms/step - loss: 0.5520 - accuracy: 0.7296 - val_loss: 0.5435 - val_accuracy: 0.7078
Epoch 8/10
20/20 [=

# HyperParameters Tunning - Keras tuner

### 1) How to select appropriate optimizer
### 2) No. of nodes in a layer
### 3) How to select No of hidden layer
### 4) All in all with one model

In [14]:
# !pip install keras-tuner -q # first install it

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 2.3 MB/s eta 0:00:00


In [10]:
import keras_tuner as kt

In [16]:
# Optimizer
def build_model(hp):
  model = Sequential()
  model.add(Dense(32, activation='relu', input_dim=8))
  model.add(Dense(1, activation='sigmoid'))
  optimizer = hp.Choice("optimizer",values = ["adam","sgd","rmsprop","adadelta"])
  model.compile(optimizer=optimizer,loss = "binary_crossentropy",metrics = ["accuracy"])
  return model

In [18]:
# Best Optimization
tuner = kt.RandomSearch(build_model, objective = "val_accuracy", max_trials = 5)

1. objective: The objective of the hyperparameter search is to maximize the validation accuracy. In this case, the "val_accuracy" is specified as the objective, which means the tuner will try to find hyperparameters that lead to higher validation accuracy.

2. max_trials: This parameter determines the total number of hyperparameter combinations to try during the search. In this case, the tuner will explore 5 different combinations of hyperparameters.

In [19]:
tuner.search(x_train,y_train, epochs = 5 , validation_data = (x_test,y_test))


Trial 4 Complete [00h 00m 03s]
val_accuracy: 0.7597402334213257

Best val_accuracy So Far: 0.7597402334213257
Total elapsed time: 00h 00m 09s


In [20]:
tuner.get_best_hyperparameters()

In [21]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [22]:
# adam is working better so we got the optimizer that we have to use

In [23]:
model = tuner.get_best_models(num_models=1)[0]

1. The line of code model = tuner.get_best_models(num_models=1)[0] is used to retrieve the best model found during the hyperparameter tuning process using Keras Tuner.

2. get_best_models(num_models=1): This method is used to retrieve the best models found during the hyperparameter tuning process. The num_models parameter specifies the number of best models to retrieve. In this case, num_models=1, meaning only the top-performing model will be returned.

3. [0]: The result of tuner.get_best_models(num_models=1) is a list containing the best model(s). Since we are interested in just the top-performing model, we use indexing [0] to access the first (and only) element in the list.

In [28]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 32)                288       
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321
Trainable params: 321
Non-trainable params: 0
_________________________________________________________________


## Lets look for neuron ?

In [35]:
def build_model(hp):
  model = Sequential()
  units = hp.Int('units',8,128, step=8)
  # 8- lower limit or min neuron
  # 128 - upper limit - max neuron
  model.add(Dense(units=units, activation ='relu', input_dim=8))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  return model

In [36]:
# Tunner object
tuner = kt.RandomSearch(build_model, objective = 'val_accuracy', max_trials=5, directory='mydir', project_name='neuron')

In [37]:
tuner.search(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

In [38]:
tuner.get_best_hyperparameters()[0].values

{'units': 112}

In [39]:
model = tuner.get_best_models(num_models=1)[0]

In [40]:
model.fit(x_train, y_train, batch_size=32, epochs=100, initial_epoch=6, validation_data=(x_test, y_test))

Epoch 7/100


array([[3.13116398e-05, 8.77908951e-06, 2.19744383e-04, 1.74236076e-04,
        4.78479114e-06, 6.60026853e-05, 6.01793317e-07, 7.52079904e-06,
        9.64353094e-05, 1.76762987e-05, 8.11001592e-05, 4.38481402e-05,
        1.17367439e-04, 2.39737827e-04, 2.93880767e-05, 5.65166665e-05,
        1.46383463e-05, 1.45976359e-04, 1.62877783e-04, 1.11026828e-04,
        8.22112779e-05, 3.64987005e-04, 1.03591636e-04, 6.82549507e-05,
        4.26429324e-06, 3.82843018e-06, 1.65569600e-06, 2.71840690e-04,
        7.69754115e-05, 6.22428479e-06, 1.67152699e-04, 4.20406577e-05,
        2.52029076e-05, 3.96193122e-04, 2.21753435e-04, 2.72824836e-05,
        1.30858625e-05, 5.38654458e-05, 3.40774830e-04, 2.05805122e-07,
        2.82144701e-05, 4.28176500e-05, 2.83356599e-07, 1.51767699e-05,
        9.36512224e-05, 1.54819536e-05, 9.38527228e-06, 1.64820249e-05,
        2.24461655e-06, 3.24737812e-06, 2.55309278e-04, 2.21548253e-05,
        2.58803513e-04, 3.49543698e-04, 2.47834629e-04, 7.034552

20/20 [==============================] - 2s 15ms/step - loss: 0.4912 - accuracy: 0.7655 - val_loss: 0.4659 - val_accuracy: 0.7727
Epoch 8/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4733 - accuracy: 0.7687 - val_loss: 0.4543 - val_accuracy: 0.7922
Epoch 9/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4655 - accuracy: 0.7704 - val_loss: 0.4457 - val_accuracy: 0.7987
Epoch 10/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4605 - accuracy: 0.7834 - val_loss: 0.4393 - val_accuracy: 0.7987
Epoch 11/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4596 - accuracy: 0.7866 - val_loss: 0.4381 - val_accuracy: 0.7922
Epoch 12/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4560 - accuracy: 0.7866 - val_loss: 0.4374 - val_accuracy: 0.7987
Epoch 13/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4526 - accuracy: 0.7850 - val_loss: 0.4381 - val_accuracy: 0.8052
Epoch 14/100
20/20 [

# How many hidden layer?

In [41]:
def build_model(hp):
  model = Sequential()
  model.add(Dense(112, activation='relu', input_dim=8))
  for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
    model.add(Dense(112,activation='relu'))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  return model

In [42]:
# Tunner object
tuner = kt.RandomSearch(build_model, objective = 'val_accuracy', max_trials=5, directory='mydir', project_name='num_layers')

In [43]:
tuner.search(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.7922077775001526

Best val_accuracy So Far: 0.8181818127632141
Total elapsed time: 00h 00m 20s


In [44]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 5}

In [45]:
model = tuner.get_best_models(num_models=1)[0]

In [46]:
model.fit(x_train, y_train, batch_size=32, epochs=100, initial_epoch=6, validation_data=(x_test, y_test))

Epoch 7/100
20/20 [==============================] - 3s 16ms/step - loss: 0.4602 - accuracy: 0.7850 - val_loss: 0.4420 - val_accuracy: 0.7662
Epoch 8/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4438 - accuracy: 0.7899 - val_loss: 0.4373 - val_accuracy: 0.7987
Epoch 9/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4278 - accuracy: 0.7932 - val_loss: 0.4373 - val_accuracy: 0.7857
Epoch 10/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4207 - accuracy: 0.8111 - val_loss: 0.4762 - val_accuracy: 0.7597
Epoch 11/100
20/20 [==============================] - 0s 6ms/step - loss: 0.4020 - accuracy: 0.8111 - val_loss: 0.4543 - val_accuracy: 0.7987
Epoch 12/100
20/20 [==============================] - 0s 7ms/step - loss: 0.3936 - accuracy: 0.8257 - val_loss: 0.4589 - val_accuracy: 0.8052
Epoch 13/100
20/20 [==============================] - 0s 6ms/step - loss: 0.3724 - accuracy: 0.8469 - val_loss: 0.4612 - val_accuracy: 0.7922
Epoch 14

# All hyperparmeter at one go

Task::

1. First task to to give number of hidden layer and then number of neurons

2.

In [11]:
def build_model(hp):
  model = Sequential()
  counter = 0

  for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
    if counter == 0: # 1st layer
      model.add(Dense(hp.Int('units' + str(i), min_value=8, max_value=128,step=8),
                      activation = hp.Choice('activation' + str(i), values = ['relu', 'tanh', 'sigmoid','elu','leaky_relu','softmax','linear']),input_dim=8))
      model.add(Dropout(hp.Choice('dropout' + str(i), values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    else :
        model.add(Dense(hp.Int('units' + str(i), min_value=8, max_value=128,step=8),
                      activation = hp.Choice('activation' + str(i), values = ['relu', 'tanh', 'sigmoid','elu','leaky_relu','softmax','linear']),))
        model.add(Dropout(hp.Choice('dropout' + str(i), values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    counter +=1
  model.add(Dense(1, activation='sigmoid')) # last layer
  model.compile(optimizer=hp.Choice('optimizer', values=['rmsprop','adam','sgd','nadam','adadelta','adagrad']), loss='binary_crossentropy', metrics=['accuracy'])
  return model

In [12]:
tuner = kt.RandomSearch(build_model, objective = 'val_accuracy',
                        max_trials=10, directory='mydir', project_name='final_1')

In [13]:
tuner.search(x_train, y_train, epochs=5, validation_data=(x_test, y_test))

Trial 10 Complete [00h 00m 02s]
val_accuracy: 0.6688311696052551

Best val_accuracy So Far: 0.7597402334213257
Total elapsed time: 00h 00m 35s


In [14]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 1,
 'units0': 32,
 'activation0': 'tanh',
 'dropout0': 0.1,
 'optimizer': 'rmsprop',
 'units1': 40,
 'activation1': 'leaky_relu',
 'dropout1': 0.9,
 'units2': 48,
 'activation2': 'elu',
 'dropout2': 0.3,
 'units3': 8,
 'activation3': 'tanh',
 'dropout3': 0.6,
 'units4': 56,
 'activation4': 'elu',
 'dropout4': 0.6,
 'units5': 48,
 'activation5': 'linear',
 'dropout5': 0.2,
 'units6': 88,
 'activation6': 'leaky_relu',
 'dropout6': 0.1,
 'units7': 80,
 'activation7': 'sigmoid',
 'dropout7': 0.3,
 'units8': 8,
 'activation8': 'tanh',
 'dropout8': 0.5}

In [15]:
model = tuner.get_best_models(num_models=1)[0]

In [16]:
model.fit(x_train, y_train, batch_size=32, epochs=200, initial_epoch=5, validation_data=(x_test, y_test))

Epoch 6/200
20/20 [==============================] - 2s 40ms/step - loss: 0.5249 - accuracy: 0.7508 - val_loss: 0.5069 - val_accuracy: 0.7468
Epoch 7/200
20/20 [==============================] - 0s 8ms/step - loss: 0.5142 - accuracy: 0.7622 - val_loss: 0.4932 - val_accuracy: 0.7597
Epoch 8/200
20/20 [==============================] - 0s 18ms/step - loss: 0.5018 - accuracy: 0.7541 - val_loss: 0.4812 - val_accuracy: 0.7662
Epoch 9/200
20/20 [==============================] - 0s 15ms/step - loss: 0.4970 - accuracy: 0.7508 - val_loss: 0.4747 - val_accuracy: 0.7792
Epoch 10/200
20/20 [==============================] - 0s 11ms/step - loss: 0.4920 - accuracy: 0.7687 - val_loss: 0.4668 - val_accuracy: 0.7792
Epoch 11/200
20/20 [==============================] - 0s 8ms/step - loss: 0.4855 - accuracy: 0.7638 - val_loss: 0.4607 - val_accuracy: 0.7922
Epoch 12/200
20/20 [==============================] - 0s 4ms/step - loss: 0.4862 - accuracy: 0.7704 - val_loss: 0.4553 - val_accuracy: 0.7987
Epoch 